##### Implementation of MoCov3 to detect dog emotions.
- Batch Size : 128
- Image resoltion : 224 X 224
- Status : Building

In [1]:
import os
import math
import shutil
import random
import time

In [14]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [10]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as torchvision_models
from torch.utils.tensorboard import SummaryWriter

In [13]:
import moco.builder
import moco.loader
import moco.LARS

In [15]:
BATCH_SIZE = 128
RESOLUTION = 224

##### Data Generator.
- Transforms yanked from the BYOL paper https://arxiv.org/abs/2006.07733.